ERA5

- mean, median, std, min, max for temperature values by year and year-month

- maps for both results


Yearly

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    # Open the dataset using memory-efficient lazy loading
    ds = xr.open_dataset(file_path, chunks={'valid_time': 500})  # Process in chunks
    
    # Extract the year from the 'valid_time' dimension
    year = ds['valid_time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the temperature data as a numpy array
    temp_data = ds['t2m'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(temp_data, axis=0)
    median = np.nanmedian(temp_data, axis=0)
    std = np.nanstd(temp_data, axis=0)
    max_ = np.nanmax(temp_data, axis=0)
    min_ = np.nanmin(temp_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from valid_time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


In [ ]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\ERA5\Temperature\yearly_temperature_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


In [18]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os

# File path
file_path = r"E:\IPMA\ERA5\Temperature\yearly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_year"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Add gridlines with labels only on bottom and left sides, no linestyle
    gridlines = ax.gridlines(draw_labels=True, linewidth=0, color='black')
    gridlines.xlabels_top = False
    gridlines.ylabels_right = False
    gridlines.xlabels_bottom = True
    gridlines.ylabels_left = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='vertical', label='°C', shrink=0.7, pad=0.1, fraction=0.02)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_temperature_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


✅ Mean temperature maps with adjusted colorbar spacing generated!


Monthly

In [20]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    # Open the dataset using memory-efficient lazy loading
    ds = xr.open_dataset(file_path, chunks={'valid_time': 500})
    
    # Extract year and month from the 'valid_time' dimension
    ds = ds.assign_coords(year=ds['valid_time'].dt.year, month=ds['valid_time'].dt.month)
    
    # Group by year and month and compute stats
    grouped = ds.groupby('month')

    for month in range(1, 13):
        if month not in grouped.groups:
            continue  # Skip if that month isn't in the file

        ds_month = grouped[month]
        temp_data = ds_month['t2m'].values

        # Compute stats over time (axis 0)
        mean = np.nanmean(temp_data, axis=0)
        median = np.nanmedian(temp_data, axis=0)
        std = np.nanstd(temp_data, axis=0)
        max_ = np.nanmax(temp_data, axis=0)
        min_ = np.nanmin(temp_data, axis=0)

        # Create a Dataset for the stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        year_val = int(ds['year'].values[0])
        stats_df['Year'] = year_val
        stats_df['Month'] = month
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])

        df_list.append(stats_df)

# Combine all months from all years
df_final = pd.concat(df_list)

# Preview (optional)
print(df_final)


                                    Mean     Median       Std        Max  \
Year Month latitude longitude                                              
1979 1     66.0     -12.00     -0.956761  -0.553314  3.134850   4.412506   
                    -11.75     -0.942214  -0.516647  3.117241   4.463287   
                    -11.50     -0.933165  -0.515823  3.115323   4.535553   
                    -11.25     -0.918887  -0.511261  3.108019   4.574615   
                    -11.00     -0.893903  -0.472977  3.086093   4.541412   
...                                  ...        ...       ...        ...   
2024 12    34.0      35.00     17.479162  17.610245  1.667804  20.843414   
                     35.25     17.684494  17.800568  1.651862  21.241119   
                     35.50     15.112276  15.030884  2.435855  21.598785   
                     35.75      9.835279   9.426178  3.429251  19.016754   
                     36.00      5.694493   5.228882  4.191517  17.098785   

           

In [21]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc


In [22]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_month"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Latitude and longitude
lat = ds['latitude']
lon = ds['longitude']

# Loop through years
for year_idx, year in enumerate(ds['Year'].values):
    # Loop through months (1 to 12)
    for month in range(1, 13):
        # Mask where this month's data is located
        month_mask = ds['Month'].isel(Year=year_idx) == month
        if not month_mask.any():
            continue  # Skip if this month doesn't exist in this year

        # Apply mask to temperature data
        data = ds[var].isel(Year=year_idx).where(month_mask)

        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}')
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        cbar = plt.colorbar(im, ax=ax, orientation='vertical', shrink=0.7, pad=0.05)
        cbar.set_label('°C')

        plt.tight_layout()
        filename = f"mean_temperature_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly mean temperature maps with Europe outline generated!")


ValueError: Dimensions {'Year'} do not exist. Expected one or more of ('Month',)

In [23]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\ERA5\Temperature\monthly_temperature_stats.nc"
output_folder = r"E:\IPMA\ERA5\Temperature\maps_month"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Temperature (°C)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'coolwarm'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']
month = int(ds['Month'].values[0])  # Only one month in the dataset

# Loop through all years
for year in ds['Year'].values:
    data = ds[var].sel(Year=year, Month=month)

    # Skip empty or all-NaN values
    if np.isnan(data).all():
        continue

    # Plotting
    fig = plt.figure(figsize=(7, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

    im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=ccrs.PlateCarree())

    ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
    ax.coastlines(resolution='10m', linewidth=0.8)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', shrink=0.7, pad=0.1)
    cbar.set_label('°C')

    # Save figure
    plt.tight_layout()
    filename = f"mean_temperature_{year}{month:02d}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Monthly temperature maps generated for all years!")


✅ Monthly temperature maps generated for all years!
